<a href="https://colab.research.google.com/github/sou-venir/Dream-nori/blob/main/%EC%BD%94%EB%9E%A9%EC%9D%84%EC%9C%84%ED%95%9C%EB%93%9C%EB%A6%BC%EB%86%80%EC%9D%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# [1] 필수 라이브러리 설치 (코랩 환경 전용)
!pip install flask-socketio flask-ngrok pyngrok openai python-socketio
!fuser -k 5000/tcp

import urllib.parse
import os, json, copy, re
from datetime import datetime
from flask import Flask, render_template_string, request, Response
from flask_socketio import SocketIO, emit
from pyngrok import ngrok
from google.colab import userdata
import openai
from google.colab import drive
import google.generativeai as genai

# =========================
# Drive & Storage
# =========================
drive.mount('/content/drive')
SAVE_PATH = '/content/drive/MyDrive/ChatData'
os.makedirs(SAVE_PATH, exist_ok=True)
DATA_FILE = os.path.join(SAVE_PATH, "save_data.json")

def save_data():
    try:
        with open(DATA_FILE, "w", encoding="utf-8") as f:
            json.dump(state, f, ensure_ascii=False, indent=2)
    except: pass

def load_data():
    if os.path.exists(DATA_FILE):
        try:
            with open(DATA_FILE, "r", encoding="utf-8") as f:
                return json.load(f)
        except: return None
    return None

# =========================
# Keys & AI setup
# =========================
ADMIN_PASSWORD = userdata.get('ADMIN_PASSWORD')

gemini_model = None
try:
    NGROK_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    client = openai.OpenAI(api_key=OPENAI_API_KEY)
    try:
        GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
        if GEMINI_API_KEY:
            genai.configure(api_key=GEMINI_API_KEY)
            gemini_model = genai.GenerativeModel('gemini-3-pro-preview')
    except: pass
    ngrok.set_auth_token(NGROK_TOKEN)
except Exception as e:
    print(f"❌ 설정 오류: {e}")

# =========================
# App
# =========================
app = Flask(__name__)
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
socketio = SocketIO(app, cors_allowed_origins="*")

# =========================
# State
# =========================
initial_state = {
    "session_title": "드림놀이",
    "theme": {"bg": "#ffffff", "panel": "#f1f3f5", "accent": "#e91e63"},
    "ai_model": "gpt-5.2",
    "admin_password": ADMIN_PASSWORD,
    "solo_mode": False,
    "session_started": False,
    "profiles": {
        "user1": {"name": "Player 1", "bio": "", "canon": "", "locked": False},
        "user2": {"name": "Player 2", "bio": "", "canon": "", "locked": False}
    },
    "pending_inputs": {},
    "ai_history": [],
    "summary": "",
    "prologue": "",
    "sys_prompt": "당신은 숙련된 TRPG 마스터입니다.",
    "lorebook": [],
    "examples": [{"q": "", "a": ""}, {"q": "", "a": ""}, {"q": "", "a": ""}]
}

saved_state = load_data()
state = saved_state if isinstance(saved_state, dict) else copy.deepcopy(initial_state)

connected_users = {"user1": None, "user2": None}
readonly_sids = set()
admin_sids = set()
typing_users = set()

# =========================
# Helpers
# =========================
def sanitize_filename(name: str) -> str:
    name = (name or "session").strip()
    name = re.sub(r'[\\/:*?"<>|]+', "_", name)
    return name[:60] or "session"

def get_export_config_only():
    return {
        "session_title": state.get("session_title", ""),
        "sys_prompt": state.get("sys_prompt", ""),
        "prologue": state.get("prologue", ""),
        "ai_model": state.get("ai_model", "gpt-5.2"),
        "examples": state.get("examples", [{"q":"","a":""},{"q":"","a":""},{"q":"","a":""}]),
        "lorebook": state.get("lorebook", []),
        "solo_mode": bool(state.get("solo_mode", False)),
        "_export_type": "dream_config_only_v1"
    }

def import_config_only(data: dict):
    allow = {"session_title","sys_prompt","prologue","ai_model","examples","lorebook","solo_mode"}
    for k in allow:
        if k in data: state[k] = copy.deepcopy(data[k])

def get_sanitized_state():
    safe = copy.deepcopy(state)
    safe["profiles"]["user1"]["bio"] = ""
    safe["profiles"]["user1"]["canon"] = ""
    safe["profiles"]["user2"]["bio"] = ""
    safe["profiles"]["user2"]["canon"] = ""
    return safe

def emit_state_to_players(save=True):
    if save: save_data()
    payload = get_sanitized_state()
    payload["pending_status"] = list(state.get("pending_inputs", {}).keys())
    payload["typing_status"] = list(typing_users)

    # User1, User2에게 전송
    if connected_users["user1"]: socketio.emit("initial_state", payload, room=connected_users["user1"])
    if connected_users["user2"]: socketio.emit("initial_state", payload, room=connected_users["user2"])
    # 관전자에게도 전송
    for rsid in readonly_sids:
        socketio.emit("initial_state", payload, room=rsid)

# (1) ✅ analyze_theme_color() 함수 전체를 이걸로 교체 (OpenAI 전용 + 들여쓰기 정상)

def analyze_theme_color(title, sys_prompt):
    prompt_text = (
        f"세션 제목: {title}\n"
        f"시스템/프롤로그: {sys_prompt[:1200]}\n\n"
        "웹 UI 컬러 팔레트 전문가입니다.\n"
        "텍스트는 항상 검정(#000000)입니다. bg/panel은 매우 밝고 대비가 높아야 합니다.\n"
        "accent는 시나리오 분위기를 대표해야 합니다.\n"
        "반드시 JSON만 반환: {\"bg\":\"#RRGGBB\",\"panel\":\"#RRGGBB\",\"accent\":\"#RRGGBB\"}"
    )
    try:
        if not client:
            return state.get("theme")

        res = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Return JSON only."},
                {"role": "user", "content": prompt_text}
            ],
            response_format={"type": "json_object"}
        )
        obj = json.loads(res.choices[0].message.content)

        out = state.get("theme", {"bg": "#ffffff", "panel": "#f1f3f5", "accent": "#e91e63"})
        for k in ("bg", "panel", "accent"):
            v = obj.get(k)
            if isinstance(v, str) and v.startswith("#") and len(v) == 7:
                out[k] = v
        return out

    except Exception as e:
        print(f"⚠️ 테마 분석 실패(OpenAI): {e}")
        return state.get("theme")

# Context / Summary
MAX_CONTEXT_CHARS_BUDGET = 14000
HISTORY_SOFT_LIMIT_CHARS = 9500
SUMMARY_MAX_CHARS = 500
TARGET_MAX_TOKENS = 1100

def build_history_block():
    history = state.get("ai_history", [])
    collected = []
    total = 0
    for msg in reversed(history):
        add_len = len(msg) + 1
        if total + add_len > HISTORY_SOFT_LIMIT_CHARS: break
        collected.append(msg)
        total += add_len
    collected.reverse()
    return collected

def would_overflow_context(extra_incoming: str) -> bool:
    sys_p = state.get("sys_prompt","")
    pro = state.get("prologue","")
    summ = state.get("summary","")
    hist = "\n".join(build_history_block())
    return (len(sys_p)+len(pro)+len(summ)+len(hist)+len(extra_incoming)+2000) > MAX_CONTEXT_CHARS_BUDGET

def auto_summary_apply():
    def run_once():
        recent_log = "\n".join(state.get("ai_history", [])[-60:])
        if not recent_log: return None
        res = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role":"user","content":f"요약해줘:\n{recent_log}"}]
        )
        return (res.choices[0].message.content or "").strip()
    try:
        s = run_once()
        if s: state["summary"] = s[:SUMMARY_MAX_CHARS]; save_data()
    except: pass

# =========================
# Routes
# =========================
@app.route("/")
def index():
    return render_template_string(HTML_TEMPLATE, theme=state.get("theme"))

@app.route("/export")
def export_config():
    cfg = get_export_config_only()
    ts = datetime.now().strftime("%Y%m%d_%H%M")
    safe_title = re.sub(r"[^A-Za-z0-9_-]+", "_", (cfg.get("session_title") or "session"))
    fname = f"{safe_title}_{ts}.json"
    data = json.dumps(cfg, ensure_ascii=False, indent=2)
    resp = Response(data, mimetype="application/json; charset=utf-8")
    resp.headers["Content-Disposition"] = "attachment; filename*=UTF-8''" + urllib.parse.quote(fname)
    return resp

@app.route("/import", methods=["POST"])
def import_config():
    try:
        if "file" not in request.files: return "파일X", 400
        file = request.files["file"]
        if file.filename == "": return "파일X", 400
        data = json.load(file)
        import_config_only(data)
        save_data()
        emit_state_to_players()
        return "OK", 200
    except Exception as e: return str(e), 500

# =========================
# Socket Logic (Fixed Session Restoration)
# =========================
@socketio.on("join_game")
def join_game(data=None):
    sid = request.sid
    saved_role = (data or {}).get("saved_role")

    # 0) 혹시 이 sid가 예전에 role로 등록돼 있으면 그대로 복구
    for role, rsid in connected_users.items():
        if rsid == sid:
            emit("assign_role", {"role": role, "mode": "player"})
            emit_state_to_players()
            return

    # 1) 저장된 역할이 있으면 "비어있을 때만" 복구 (강탈 X)
    if saved_role in ("user1", "user2"):
        if connected_users[saved_role] is None:
            connected_users[saved_role] = sid
            emit("assign_role", {"role": saved_role, "mode": "player"})
            emit_state_to_players()
            return
        else:
            print("join_game: slot_taken", saved_role, "existing_sid=", connected_users.get(saved_role), "new_sid=", sid)
            readonly_sids.add(sid)
            emit("assign_role", {"role": "readonly", "mode": "readonly", "reason": "slot_taken"})
            emit_state_to_players()
            return

    # 2) 신규 접속: 빈 자리 배정
    if connected_users["user1"] is None:
        connected_users["user1"] = sid
        emit("assign_role", {"role": "user1", "mode": "player"})
        emit_state_to_players()
        return

    if connected_users["user2"] is None:
        connected_users["user2"] = sid
        emit("assign_role", {"role": "user2", "mode": "player"})
        emit_state_to_players()
        return

    # 3) 만석이면 readonly
    readonly_sids.add(sid)
    emit("assign_role", {"role": "readonly", "mode": "readonly", "reason": "full"})
    emit_state_to_players()


# ✅ disconnect는 이 블록 하나만 남겨. (중복된 disconnect 데코레이터/함수는 삭제)
@socketio.on("disconnect")
def on_disconnect():
    sid = request.sid
    admin_sids.discard(sid)

    for role in ("user1", "user2"):
        if connected_users[role] == sid:
            connected_users[role] = None
            typing_users.discard(role)
            state.get("pending_inputs", {}).pop(role, None)

    readonly_sids.discard(sid)
    save_data()
    emit_state_to_players()

@socketio.on("start_typing")
def start_typing(data):
    uid = data.get("uid")
    if uid in ("user1","user2") and connected_users.get(uid) == request.sid:
        typing_users.add(uid)
        emit_state_to_players(save=False)

@socketio.on("stop_typing")
def stop_typing(data):
    uid = data.get("uid")
    if uid in ("user1","user2"):
        typing_users.discard(uid)
        emit_state_to_players(save=False)

@socketio.on("check_admin")
def check_admin(data):
    ok = str(data.get("password")) == str(state.get("admin_password"))
    if ok: admin_sids.add(request.sid)
    emit("admin_auth_res", {"success": ok})

@socketio.on("save_master_base")
def save_master_base(data):
    state["session_title"] = (data.get("title", state["session_title"]) or "")[:30]
    state["sys_prompt"] = (data.get("sys", state["sys_prompt"]) or "")[:4000]
    state["prologue"] = (data.get("pro", state["prologue"]) or "")[:1000]
    state["summary"] = (data.get("sum", state["summary"]) or "")[:SUMMARY_MAX_CHARS]
    state["ai_model"] = data.get("model", state.get("ai_model","gpt-5.2"))
    state["solo_mode"] = bool(data.get("solo_mode", state.get("solo_mode", False)))
    save_data()
    emit_state_to_players()

@socketio.on("theme_analyze_request")
def theme_analyze_request(_=None):
    if not (state.get("sys_prompt","").strip() and state.get("prologue","").strip()):
        return
    # prologue까지 합쳐서 분석 품질 올리기
    combined = state.get("sys_prompt","") + "\n\n[PROLOGUE]\n" + state.get("prologue","")
    state["theme"] = analyze_theme_color(state.get("session_title",""), combined)
    save_data()
    emit_state_to_players()


@socketio.on("save_examples")
def save_examples(data):
    out = []
    for i in range(3):
        ex = data[i] if i < len(data) else {"q":"","a":""}
        out.append({"q": (ex.get("q","") or "")[:500], "a": (ex.get("a","") or "")[:500]})
    state["examples"] = out
    save_data()
    emit_state_to_players()

@socketio.on("update_profile")
def update_profile(data):
    uid = data.get("uid")
    if uid not in ("user1","user2"): return
    if connected_users.get(uid) != request.sid: return
    if state["profiles"][uid].get("locked"): return
    name = (data.get("name") or "").strip()
    if not name: return
    state["profiles"][uid]["name"] = name[:12]
    state["profiles"][uid]["bio"] = (data.get("bio") or "")[:200]
    state["profiles"][uid]["canon"] = (data.get("canon") or "")[:350]
    state["profiles"][uid]["locked"] = True
    save_data()
    emit_state_to_players()

@socketio.on("start_session")
def start_session(_=None):
    if request.sid not in admin_sids: return
    if state.get("session_started"): return
    state["session_started"] = True
    save_data()
    emit_state_to_players()
    emit("status_update", {"msg": "✅ 세션이 시작되었습니다!"}, broadcast=True)

@socketio.on("add_lore")
def add_lore(data):
    idx = int(data.get("index", -1))
    title = (data.get("title","") or "")[:20]
    triggers = (data.get("triggers","") or "")
    content = (data.get("content","") or "")[:400]
    item = {"title": title, "triggers": triggers, "content": content}
    state.setdefault("lorebook", [])
    if (idx < 0 or idx >= len(state["lorebook"])) and len(state["lorebook"]) >= 20:
        emit("status_update", {"msg": "⚠️ 키워드북은 최대 20개까지 가능합니다."})
        return
    if 0 <= idx < len(state["lorebook"]): state["lorebook"][idx] = item
    else: state["lorebook"].append(item)
    save_data()
    emit_state_to_players()

@socketio.on("del_lore")
def del_lore(data):
    try: state["lorebook"].pop(int(data.get("index"))); save_data(); emit_state_to_players()
    except: pass

@socketio.on("reorder_lore")
def reorder_lore(data):
    try:
        f, t = int(data.get("from")), int(data.get("to"))
        state["lorebook"].insert(t, state["lorebook"].pop(f))
        save_data(); emit_state_to_players()
    except: pass

# (서버) reset_session 이벤트를 아래로 교체
@socketio.on("reset_session")
def reset_session(data):
    if str(data.get("password")) != str(state.get("admin_password")):
        emit("status_update", {"msg": "❌ 비밀번호가 일치하지 않습니다."})
        return

    # 1) 상태 전부 초기화(화면에 보이는 모든 입력값이 비게)
    state["session_title"] = "드림놀이"
    state["theme"] = {"bg": "#ffffff", "panel": "#f1f3f5", "accent": "#e91e63"}
    state["ai_model"] = "gpt-5.2"
    state["solo_mode"] = False
    state["session_started"] = False

    state["profiles"]["user1"] = {"name": "Player 1", "bio": "", "canon": "", "locked": False}
    state["profiles"]["user2"] = {"name": "Player 2", "bio": "", "canon": "", "locked": False}

    state["pending_inputs"] = {}
    typing_users.clear()

    state["ai_history"] = []
    state["summary"] = ""
    state["prologue"] = ""
    state["sys_prompt"] = "당신은 숙련된 TRPG 마스터입니다."

    state["lorebook"] = []
    state["examples"] = [{"q": "", "a": ""}, {"q": "", "a": ""}, {"q": "", "a": ""}]

    save_data()
    emit_state_to_players()
    emit("status_update", {"msg": "🧹 세션이 완전히 초기화되었습니다."}, broadcast=True)

def record_pending(uid, text):
    state.setdefault("pending_inputs", {})
    state["pending_inputs"][uid] = {"text": (text or "")[:600], "ts": datetime.now().isoformat()}
    save_data()

def both_ready():
    if state.get("solo_mode"): return "user1" in state.get("pending_inputs", {})
    return "user1" in state.get("pending_inputs", {}) and "user2" in state.get("pending_inputs", {})

def trigger_ai_from_pending():
    pending = state.get("pending_inputs", {})
    p1_text = pending.get("user1", {}).get("text", "(스킵)")
    p2_text = pending.get("user2", {}).get("text", "(스킵)")
    p1_name = state["profiles"]["user1"].get("name","Player 1")
    p2_name = state["profiles"]["user2"].get("name","Player 2")

    merged = f"{p1_text}\n{p2_text}"
    active_context = []
    for l in state.get("lorebook", []):
        triggers = [t.strip() for t in (l.get("triggers","")).split(",") if t.strip()]
        if any(t in merged for t in triggers):
            active_context.append(f"[{l.get('title','')}]: {l.get('content','')}")
    active_context = active_context[:3]

    system_content = f"{state.get('sys_prompt','')}\n\n[Summary]\n{state.get('summary','')}\n\n[Lore]\n" + "\n".join(active_context)
    if would_overflow_context(system_content + merged):
        auto_summary_apply()
        system_content = f"{state.get('sys_prompt','')}\n\n[Summary]\n{state.get('summary','')}\n\n[Lore]\n" + "\n".join(active_context)

    round_block = f"--- [ROUND INPUT] ---\n<{p1_name}>: {p1_text}\n<{p2_name}>: {p2_text}\n--- [INSTRUCTION] ---\n두 행동은 동시간대입니다. 통합하여 2000자 내외로 서술하세요."

    messages = [{"role":"system","content":system_content}]
    for ex in state.get("examples", []):
        if ex.get("q"): messages.extend([{"role":"user","content":ex["q"]}, {"role":"assistant","content":ex["a"]}])
    for h in build_history_block():
        messages.append({"role": "assistant" if h.startswith("**AI**") else "user", "content": h})
    messages.append({"role":"user","content": round_block})

    current_model = state.get("ai_model","gpt-5.2")
    socketio.emit("status_update", {"msg": f"🤔 {current_model} 집필 중..."})

    ai_response = ""
    try:
        if "gemini" in current_model.lower() and gemini_model:
            from google.generativeai.types import HarmCategory, HarmBlockThreshold
            safe = {HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE}
            prompt = system_content + "\n" + "\n".join(build_history_block()) + "\n" + round_block
            ai_response = gemini_model.generate_content(prompt, safety_settings=safe).text
        elif client:
            res = client.chat.completions.create(model=current_model, messages=messages, max_tokens=TARGET_MAX_TOKENS)
            ai_response = res.choices[0].message.content
        else:
            ai_response = "API Key Error."
    except Exception as e:
        ai_response = f"Error: {e}"

    state["ai_history"].append(f"**Round**: {p1_name}: {p1_text} / {p2_name}: {p2_text}")
    state["ai_history"].append(f"**AI**: {ai_response}")
    state["pending_inputs"] = {}
    save_data()

    socketio.emit("ai_typewriter_event", {"content": ai_response})
    emit_state_to_players()

@socketio.on("client_message")
def client_message(data):
    uid, text = data.get("uid"), (data.get("text") or "").strip()
    if uid not in ("user1","user2") or not state.get("session_started"): return
    record_pending(uid, text)
    typing_users.discard(uid)
    emit_state_to_players()
    if both_ready(): trigger_ai_from_pending()
    else:
        other = "user2" if uid == "user1" else "user1"
        nm = state["profiles"][other].get("name", other)
        socketio.emit("status_update", {"msg": f"⏳ {nm}님 입력 대기... (스킵 가능)"})

@socketio.on("skip_turn")
def skip_turn(data):
    uid = data.get("uid")
    if uid not in ("user1","user2") or not state.get("session_started"): return
    record_pending(uid, "(스킵)")
    typing_users.discard(uid)
    emit_state_to_players()
    if both_ready(): trigger_ai_from_pending()
    else:
        other = "user2" if uid == "user1" else "user1"
        nm = state["profiles"][other].get("name", other)
        socketio.emit("status_update", {"msg": f"⏳ {nm}님 입력 대기... (스킵 가능)"})

# =========================
# HTML Template
# =========================
HTML_TEMPLATE = r"""<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>드림놀이</title>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.0.1/socket.io.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/dompurify@3.1.6/dist/purify.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/sortablejs@1.15.2/Sortable.min.js"></script>

  <style>
    :root{
      --bg: {{ theme.bg if theme else '#ffffff' }};
      --panel: {{ theme.panel if theme else '#f1f3f5' }};
      --accent: {{ theme.accent if theme else '#e91e63' }};
    }
    html,body{height:100%;margin:0;overflow:hidden;}
    body{font-family:Pretendard,sans-serif;display:flex;background:var(--bg);color:#000;}
    #main{flex:1;display:flex;flex-direction:column;height:100vh;border-right:1px solid rgba(0,0,0,0.05);min-width:0;}
    #chat-window{flex:1;overflow-y:auto;padding:30px 10%;display:flex;flex-direction:column;gap:15px;scroll-behavior:smooth;}
    #chat-content{display:flex;flex-direction:column;gap:15px;padding-bottom:20px;}
    #sidebar{width:320px;height:100vh;background:var(--panel);display:flex;flex-direction:column;overflow:hidden;}
    #sidebar-body{padding:20px;overflow-y:auto;flex:1;min-height:0;display:flex;flex-direction:column;gap:12px;}
    #sidebar-footer{padding:12px 20px 16px;border-top:1px solid rgba(0,0,0,0.06);background:var(--panel);}

    textarea,input,select{background:var(--bg)!important;border:1px solid rgba(0,0,0,0.1)!important;border-radius:10px;padding:10px;width:100%;box-sizing:border-box;resize:none!important;}
    #msg-input{background:var(--panel)!important;border:1px solid rgba(0,0,0,0.15)!important;height:80px;}
    button{cursor:pointer;border:none;border-radius:8px;background:var(--accent);padding:10px;font-weight:bold;color:#fff;}
    button:hover{opacity:.85;}
    .btn-reset{background:#ff4444!important;}
    .master-btn{width:100%;background:transparent!important;color:#999!important;border:1px solid #ddd!important;padding:10px!important;border-radius:10px;font-weight:800;}

    /* [추가] 말풍선 좌우 정렬 스타일 */
    .bubble{padding:12px 18px;border-radius:15px;max-width:85%;line-height:1.6;font-size:14px;white-space:pre-wrap;position:relative;word-wrap:break-word;}

    .align-left { align-self: flex-start; background: rgba(0,0,0,0.04); color: #000; border-top-left-radius: 2px; }
    .align-left .name-tag { color: #666; font-size: 11px; font-weight: bold; margin-bottom: 4px; }

    .align-right { align-self: flex-end; background: var(--accent); color: #fff !important; border-top-right-radius: 2px; }
    .align-right .name-tag { color: rgba(255,255,255,0.8); text-align: right; font-size: 11px; font-weight: bold; margin-bottom: 4px; }
    .align-right p, .align-right span { color: #fff !important; }

    .center-ai{align-self:center;background:var(--panel)!important;border-left:5px solid var(--accent);width:100%;max-width:800px;box-shadow:0 4px 15px rgba(0,0,0,0.05);}
    .center-ai .name-tag { font-weight:900; color:var(--accent); }

    .typing-anim{animation:blink 1.4s infinite;}
    @keyframes blink{50%{opacity:.45;}}

    /* [추가] 글자수 카운터 스타일 */
    .char-cnt { font-size: 10px; color: #888; text-align: right; margin-top: 2px; }

    /* [요청] 입력창 높이 수정 */
    #p-bio { height: 160px !important; }   /* 캐릭터 설정 */
    #p-canon { height: 180px !important; } /* 관계 설정 (제일 크게) */

    /* modal */
    #admin-modal{display:none;position:fixed;z-index:10000;left:0;top:0;width:100vw;height:100vh;background:rgba(0,0,0,0.6);backdrop-filter:blur(5px);align-items:center;justify-content:center;padding:24px;box-sizing:border-box;}
    .modal-content{width:100%;max-width:1200px;height:min(85vh,900px);background:#fff;border-radius:16px;display:flex;flex-direction:column;overflow:hidden;min-height:0;box-shadow:0 20px 60px rgba(0,0,0,0.3);}
    .modal-header{height:60px;flex:0 0 60px;display:flex;justify-content:space-between;align-items:center;padding:0 20px;background:#f8f9fa;border-bottom:1px solid #eee;box-sizing:border-box;}
    .tab-group{display:flex;gap:10px;height:100%;align-items:center;}
    .tab-btn{border:none;background:none!important;padding:0 14px;height:100%;font-size:14px;font-weight:700;color:#777;cursor:pointer;position:relative;}
    .tab-btn.active{color:var(--accent);}
    .tab-btn.active::after{content:"";position:absolute;bottom:0;left:0;width:100%;height:3px;background:var(--accent);}
    .close-btn{width:32px;height:32px;border-radius:50%;border:none;background:#eee;color:#000;font-size:16px;font-weight:800;cursor:pointer;padding:0;}
    .modal-body{flex:1;display:flex;overflow:hidden;min-height:0;}
    .tab-content{display:none;width:100%;height:100%;flex-direction:row;min-height:0;}
    .tab-content.active{display:flex;}
    .editor-side{flex:1.25;padding:20px;display:flex;flex-direction:column;gap:12px;overflow-y:auto;border-right:1px solid #f0f0f0;min-height:0;box-sizing:border-box;}
    .list-side{flex:.75;padding:20px;background:#fafafa;display:flex;flex-direction:column;gap:12px;overflow-y:auto;min-height:0;box-sizing:border-box;}
    .editor-side label,.list-side label{font-size:12px;font-weight:800;color:#999;text-transform:uppercase;}
    .save-btn{background:var(--accent);color:#fff;padding:14px;border-radius:10px;font-weight:800;border:none;}
    .fill-textarea{flex:1;min-height:260px;}
    .short-textarea{flex:none;height:160px;}
    .ex-block{background:#fff;border:1px solid #eee;padding:12px;border-radius:10px;display:flex;flex-direction:column;gap:8px;}
    .ex-block textarea{height:130px!important;}
    textarea::placeholder{color:#9aa0a6;font-weight:700;}

    /* tags */
    #tag-container{display:flex;flex-wrap:wrap;gap:8px;padding:10px;border:1px solid rgba(0,0,0,0.12);border-radius:10px;background:var(--bg);align-items:center;min-height:44px;box-sizing:border-box;}
    .tag-chip{display:inline-flex;align-items:center;gap:6px;padding:6px 10px;border-radius:999px;background:rgba(0,0,0,0.06);border:1px solid rgba(0,0,0,0.08);font-size:12px;font-weight:700;user-select:none;}
    .tag-chip button{background:transparent!important;border:none;padding:0;cursor:pointer;color:#444;font-weight:900;}
    #tag-input{border:none!important;outline:none!important;background:transparent!important;width:220px!important;min-width:120px;padding:6px 8px!important;}

    /* lore list */
    .lore-row{display:flex;align-items:center;gap:8px;padding:6px 8px;border-radius:10px;background:rgba(0,0,0,0.03);border:1px solid rgba(0,0,0,0.06);}
    .drag-handle{cursor:grab;color:#999;font-size:16px;user-select:none;}
    .lore-main{flex:1;min-width:0;}
    .lore-title{font-weight:800;font-size:13px;}
    .lore-trg{font-size:11px;color:#666;}
    .lore-actions{display:flex;gap:6px;}
    .mini-btn{padding:3px 7px;font-size:11px;border-radius:8px;}
    .mini-edit{background:#44aaff!important;}
    .mini-del{background:#ff4444!important;}
    /* profile overlay fix (A: 입력 3개만 덮기) */
#profile-wrap {
  position: relative; /* 오버레이 가두기 */
  z-index: 0;         /* 스태킹 컨텍스트 생성 (뚫림 방지) */
  overflow: hidden;   /* 튀어나감 방지 */
  border-radius: 12px;
  /* 여백이 없으면 오버레이가 너무 빡빡해 보일 수 있음. 살짝 줌 */
  padding: 2px;
}

#profile-lock-overlay {
  position: absolute;
  inset: 0;
  z-index: 100;

  /* 여기를 투명으로 변경! */
  background: transparent;

  border-radius: 12px;
  cursor: not-allowed; /* 마우스 올리면 '금지' 표시는 뜨게 유지 */
}
    /* Override */
    body, #main, #sidebar, #admin-modal, .modal-content, h1,h2,h3,h4,h5,h6,p,span,div,label,input,textarea,select,option{ color:#000 !important; }
    textarea::placeholder, input::placeholder{ color: rgba(0,0,0,0.45) !important; font-weight:700; }
    .tab-btn{ color:#000 !important; opacity:0.7; } .tab-btn.active{ opacity:1; }
    #status, .name-tag, #role-display{ color:#000 !important; }
    a, a:visited { color:#000 !important; text-decoration:none; }
    input:disabled, textarea:disabled { color: #000 !important; -webkit-text-fill-color: #000 !important; opacity: 1 !important; background: rgba(0,0,0,0.04) !important; }
  </style>
</head>

<body>
  <div id="main">
    <div id="chat-window"><div id="chat-content"></div></div>

    <div id="input-area" style="padding:20px;background:var(--bg);">
      <div id="status" style="font-size:12px;margin-bottom:5px;color:var(--accent);font-weight:bold;">대기 중</div>

      <div style="display:flex;gap:10px;align-items:stretch;">
        <textarea id="msg-input" maxlength="600" placeholder="행동을 입력하세요..."></textarea>
        <div style="display:flex;flex-direction:column;gap:8px;width:110px;">
          <button id="send-btn" onclick="send()" style="width:110px;">전송</button>
          <button id="skip-btn" onclick="skipTurn()" style="width:110px;background:transparent;color:#666;border:1px solid rgba(0,0,0,0.2);padding:6px 10px;font-size:12px;font-weight:800;">스킵</button>
        </div>
      </div>
    </div>
  </div>

  <div id="sidebar">
    <div id="sidebar-body">
      <h3>설정 <span id="role-badge" style="font-size:12px;color:var(--accent)"></span></h3>
      <div id="role-display" style="padding:10px;background:rgba(0,0,0,0.05);border-radius:8px;font-weight:800;color:#555;">접속 중...</div>

      <!-- 여기가 핵심: 잠금 기능을 위한 포장지 -->
      <div id="profile-wrap">
        <div id="profile-lock-overlay" style="display:none;"></div>

        <input type="text" id="p-name" maxlength="12" placeholder="이름">

        <div>
          <textarea id="p-bio" maxlength="200" oninput="upCnt(this)" placeholder="캐릭터 설정 (최대 200자)"></textarea>
          <div id="cnt-p-bio" class="char-cnt">0/200</div>
        </div>

        <div>
          <textarea id="p-canon" maxlength="350" oninput="upCnt(this)" placeholder="관계 설정 (최대 350자)"></textarea>
          <div id="cnt-p-canon" class="char-cnt">0/350</div>
        </div>
      </div>
      <!-- //profile-wrap 끝 -->

      <button onclick="saveProfile()" id="ready-btn">설정 저장</button>
    </div>

    <div id="sidebar-footer">
      <button class="master-btn" onclick="requestAdmin()">마스터 설정</button>
    </div>
  </div>

  <div id="admin-modal">
    <div class="modal-content">
      <div class="modal-header">
        <div class="tab-group">
          <button class="tab-btn active" onclick="openTab(event,'t-base')">엔진</button>
          <button class="tab-btn" onclick="openTab(event,'t-story')">서사</button>
          <button class="tab-btn" onclick="openTab(event,'t-ex')">학습</button>
          <button class="tab-btn" onclick="openTab(event,'t-lore')">키워드</button>
        </div>
        <button onclick="closeModal(true)" class="close-btn">✕</button>
      </div>

      <div class="modal-body">
        <!-- 엔진 -->
        <div id="t-base" class="tab-content active">
          <div class="editor-side" style="display:flex;flex-direction:column;min-height:0;">
            <label>시스템 프롬프트 (최대 4000자)</label>
            <textarea id="m-sys" class="fill-textarea" maxlength="4000" oninput="upCnt(this)" style="flex:1;min-height:0;"></textarea>
            <div id="cnt-m-sys" class="char-cnt">0/4000</div>
            <button onclick="saveMaster()" class="save-btn" style="flex:0 0 auto;">저장</button>
          </div>
          <div class="list-side" style="display:flex;flex-direction:column;min-height:0;">
            <label>세션 설정 / 백업</label>
            <div style="display:flex;gap:6px;">
              <a href="/export" target="_blank" style="flex:1;">
                <button style="width:100%;background:#444!important;" class="mini-btn">백업 저장</button>
              </a>
              <button onclick="document.getElementById('import-file').click()" style="flex:1;background:#666!important;" class="mini-btn">복원</button>
              <input type="file" id="import-file" style="display:none;" accept=".json" onchange="uploadSessionFile(this)">
            </div>
            <textarea id="m-sum" class="short-textarea" maxlength="500" placeholder="현재 상황 요약(내부 기억용)"></textarea>
            <label>AI 모델 선택</label>
            <select id="m-ai-model">
              <option value="gpt-5.2">OpenAI GPT-5.2</option>
              <option value="gpt-4o">OpenAI GPT-4o</option>
              <option value="gemini-3-pro-preview">Google Gemini 3 Pro</option>
            </select>
            <label>1인 플레이 모드 (테스트용)</label>
            <select id="m-solo">
              <option value="false">사용 안 함(2인)</option>
              <option value="true">사용(1인)</option>
            </select>
            <div style="margin-top:auto; display:flex; gap:8px;">
              <button id="start-session-btn" onclick="startSession()" class="save-btn" style="background:#444!important; display:none; flex:1;">세션 시작</button>
              <button id="reset-session-btn" onclick="sessionReset()" class="btn-reset" style="display:none; flex:1;">세션 초기화</button>
            </div>
          </div>
        </div>
        <!-- 서사 -->
        <div id="t-story" class="tab-content">
          <div class="editor-side">
            <label>세션 제목 (최대 30자)</label>
            <input type="text" id="m-title" maxlength="30">
            <label>프롤로그 (최대 1000자)</label>
            <textarea id="m-pro" class="fill-textarea" maxlength="1000" oninput="upCnt(this)"></textarea>
            <div id="cnt-m-pro" class="char-cnt">0/1000</div>
            <button onclick="saveMaster()" class="save-btn">저장</button>
          </div>
          <div class="list-side">
            <label>안내</label>
            <p style="font-size:13px;color:#666;">프롬프트와 프롤로그가 모두 존재하면 모달 닫기 시 테마가 자동 분석됩니다.</p>
          </div>
        </div>
        <!-- 학습 -->
        <div id="t-ex" class="tab-content">
          <div class="editor-side">
            <label>말투 학습(예시 대화 3쌍, 각 500자)</label>
            <div class="ex-block">
              <label>Example 1</label>
              <textarea id="ex-q-0" maxlength="500" placeholder="질문"></textarea>
              <textarea id="ex-a-0" maxlength="500" placeholder="답변"></textarea>
            </div>
            <div class="ex-block">
              <label>Example 2</label>
              <textarea id="ex-q-1" maxlength="500" placeholder="질문"></textarea>
              <textarea id="ex-a-1" maxlength="500" placeholder="답변"></textarea>
            </div>
            <div class="ex-block">
              <label>Example 3</label>
              <textarea id="ex-q-2" maxlength="500" placeholder="질문"></textarea>
              <textarea id="ex-a-2" maxlength="500" placeholder="답변"></textarea>
            </div>
            <button onclick="saveExamples()" class="save-btn">저장</button>
          </div>
        </div>
        <!-- 키워드 (UI 복구됨) -->
        <div id="t-lore" class="tab-content">
          <div class="editor-side">
             <h3 style="margin:0 0 10px 0; font-size:16px;">키워드/설정 추가</h3>

             <div>
               <label style="font-size:12px; font-weight:bold; color:#888;">키워드 이름 (20자)</label>
               <input type="text" id="kw-t" maxlength="20" placeholder="예: 마법학교, 절대반지">
             </div>

             <div>
               <label style="font-size:12px; font-weight:bold; color:#888;">트리거 (엔터/스페이스로 추가)</label>
               <div id="tag-container" onclick="focusTagInput()">
                 <input type="text" id="tag-input" placeholder="대화에 이 단어가 나오면 AI가 기억합니다">
               </div>
               <input type="hidden" id="tag-hidden" value="">
             </div>

             <div style="flex:1; display:flex; flex-direction:column;">
               <label style="font-size:12px; font-weight:bold; color:#888; margin-bottom:5px;">상세 설정 내용</label>
               <textarea id="kw-c" class="fill-textarea" maxlength="400" oninput="upCnt(this)" placeholder="이 키워드에 대한 설명을 자유롭게 적어주세요."></textarea>
               <div id="cnt-kw-c" class="char-cnt">0/400</div>
             </div>

             <input type="hidden" id="kw-index" value="-1">
             <button onclick="addLoreWithTags()" class="save-btn">저장 / 수정 완료</button>
          </div>

          <div class="list-side">
            <div style="display:flex; justify-content:space-between; align-items:center; margin-bottom:10px;">
              <label style="font-weight:bold;">저장된 키워드</label>
              <span style="font-size:11px; color:#888;">드래그로 우선순위 변경</span>
            </div>
            <div id="lore-list" style="flex:1; overflow-y:auto; display:flex; flex-direction:column; gap:8px;"></div>
          </div>
        </div> <!-- /#t-lore -->
      </div> <!-- /.modal-body -->
    </div>   <!-- /.modal-content -->
  </div>     <!-- /#admin-modal -->

<script>
  const socket = io();
  let gState = null;
  let myRole = null;
  let tags = [];
  let sortable = null;
  let isTypewriter = false;

  function mdToSafeHtml(mdText){
    const raw = marked.parse(mdText || "");
    return DOMPurify.sanitize(raw, {USE_PROFILES: {html: true}});
  }

  // [추가] 글자수 카운터
  function upCnt(el){
    const id = "cnt-"+el.id;
    const c = document.getElementById(id);
    if(c) c.innerText = el.value.length + "/" + el.getAttribute("maxlength");
  }

  // tags restored!
  function focusTagInput(){ document.getElementById('tag-input')?.focus(); }
  function syncHidden(){ document.getElementById('tag-hidden').value = tags.join(','); }
  function renderTags(){
    const container = document.getElementById('tag-container');
    const input = document.getElementById('tag-input');
    if(!container || !input) return;
    [...container.querySelectorAll('.tag-chip')].forEach(el=>el.remove());
    tags.forEach((t, idx)=>{
      const chip = document.createElement('span');
      chip.className='tag-chip';
      chip.innerHTML = `<span>${t}</span>`;
      const x = document.createElement('button');
      x.textContent='×';
      x.onclick=(e)=>{e.stopPropagation(); tags.splice(idx,1); renderTags();};
      chip.appendChild(x);
      container.insertBefore(chip, input);
    });
    syncHidden();
  }
  function addTag(raw){
    const t = (raw||"").trim();
    if(!t) return;
    if(t.length>20) return alert("트리거는 20자 이내로 입력해주세요.");
    if(tags.length>=5) return alert("트리거는 최대 5개까지만 가능합니다.");
    if(tags.includes(t)) return;
    tags.push(t); renderTags();
  }
  function loadTagsFromString(s){
    tags=[]; (s||"").split(',').map(x=>x.trim()).filter(Boolean).forEach(x=>{ if(!tags.includes(x)) tags.push(x); });
    renderTags();
  }
  document.addEventListener('keydown', (e)=>{
    const ti = document.getElementById('tag-input');
    if(ti && document.activeElement===ti){
      if(e.key==='Enter' || e.key===' ' || e.key===','){ e.preventDefault(); addTag(ti.value); ti.value=''; }
      if(e.key==='Backspace' && ti.value==='' && tags.length>0){ tags.pop(); renderTags(); }
    }
  });
  function clearLoreEditor(){
    document.getElementById('kw-t').value="";
    document.getElementById('kw-c').value="";
    document.getElementById('kw-index').value="-1";
    tags=[]; renderTags();
    document.getElementById('tag-input').value="";
    upCnt(document.getElementById('kw-c'));
  }

  socket.on('connect', () => {
    const savedRole = localStorage.getItem('dream_role');
    socket.emit('join_game', { saved_role: savedRole });
  });

  socket.on('reload_signal', ()=> window.location.reload());

  socket.on('assign_role', payload=>{
  myRole = payload.role;

  // 서버가 readonly로 보냈으면, 브라우저 저장 역할을 지워서 다음 접속 때 꼬임 방지
  if(payload.mode === 'readonly' || myRole === 'readonly'){
    localStorage.removeItem('dream_role');
  } else {
    localStorage.setItem('dream_role', myRole);
  }

  const roleEl = document.getElementById('role-display');
  const badgeEl = document.getElementById('role-badge');

  if(payload.mode === 'readonly'){
  roleEl.innerText = payload.reason === "slot_taken"
    ? "읽기 전용(자리가 이미 사용 중)"
    : "읽기 전용(만석)";
  if(badgeEl) badgeEl.innerText = "";
  document.getElementById('msg-input').disabled = true;
  document.getElementById('send-btn').disabled = true;
  document.getElementById('skip-btn').disabled = true;
  return;
}

  const who = (myRole==='user1') ? "Player 1" : "Player 2";
  roleEl.innerText = who + " (당신)";
  if(badgeEl) badgeEl.innerText = (myRole==='user1') ? "(P1)" : "(P2)";
});

  socket.on('status_update', d=>{
    const s = document.getElementById('status');
    s.innerHTML = d.msg;
    s.style.color = d.msg.includes('❌') ? 'red' : 'var(--accent)';
  });

  socket.on('ai_typewriter_event', d=>{
    isTypewriter = true;
    const cc = document.getElementById('chat-content');
    const wrap = document.createElement('div');
    // AI: Center style
    wrap.className = 'bubble center-ai';
    wrap.innerHTML = `<div class="name-tag">AI</div>`;
    cc.appendChild(wrap);

    const full = d.content || "";
    let i = 0;
    const tick = setInterval(()=>{
      i += 5;
      if(i > full.length) i = full.length;
      wrap.innerHTML = `<div class="name-tag">AI</div>` + mdToSafeHtml(full.slice(0, i));
      document.getElementById('chat-window').scrollTop = document.getElementById('chat-window').scrollHeight;
      if(i >= full.length){
        clearInterval(tick);
        isTypewriter = false;
        refreshUI();
      }
    }, 20);
  });

  socket.on('admin_auth_res', d=>{
    const ssb = document.getElementById('start-session-btn');
    const rsb = document.getElementById('reset-session-btn');
    if(d.success){
      document.getElementById('admin-modal').style.display='flex';
      document.querySelectorAll('.tab-content').forEach(c=>c.classList.remove('active'));
      document.querySelectorAll('.tab-btn').forEach(b=>b.classList.remove('active'));
      document.getElementById('t-base').classList.add('active');
      document.querySelector('.tab-btn').classList.add('active');
      if(ssb) ssb.style.display = 'block';
      if(rsb) rsb.style.display = 'block';
      refreshUI();
    } else {
      if(ssb) ssb.style.display = 'none';
      if(rsb) rsb.style.display = 'none';
      alert("비밀번호가 일치하지 않습니다.");
    }
  });

  socket.on('initial_state', data=>{
    gState = data;
    if(data.theme){
      const root = document.documentElement.style;
      root.setProperty('--bg', data.theme.bg);
      root.setProperty('--panel', data.theme.panel);
      root.setProperty('--accent', data.theme.accent);
    }
    if(!isTypewriter) refreshUI();
  });

  function requestAdmin(){
    const pw = prompt("관리자 비밀번호를 입력하세요:");
    if(pw) socket.emit('check_admin', {password: pw});
  }

  function closeModal(maybeAnalyze){
    document.getElementById('admin-modal').style.display='none';
    if(maybeAnalyze){
      const sys = (document.getElementById('m-sys').value||"").trim();
      const pro = (document.getElementById('m-pro').value||"").trim();
      if(sys && pro) socket.emit('theme_analyze_request');
    }
  }
  function openTab(evt,id){
    document.querySelectorAll('.tab-content').forEach(c=>c.classList.remove('active'));
    document.querySelectorAll('.tab-btn').forEach(b=>b.classList.remove('active'));
    document.getElementById(id).classList.add('active');
    evt.currentTarget.classList.add('active');
  }

  // typing emit
  const msgInput = document.getElementById('msg-input');
  let typingTimer = null;
  msgInput.addEventListener('input', ()=>{
    if(!myRole || myRole==='readonly') return;
    socket.emit('start_typing', {uid: myRole});
    clearTimeout(typingTimer);
    typingTimer = setTimeout(()=> socket.emit('stop_typing', {uid: myRole}), 1200);
  });

function refreshUI(){
    if(!gState) return;

    const msg = document.getElementById('msg-input');
    const sendBtn = document.getElementById('send-btn');
    const skipBtn = document.getElementById('skip-btn');

    if(myRole==='readonly'){
      msg.disabled=true; sendBtn.disabled=true; skipBtn.disabled=true;
    } else {
      if(gState.session_started){
        msg.disabled=false; msg.placeholder="행동을 입력하세요...";
      } else {
        msg.disabled=true; msg.placeholder="프로필 확정 후 마스터가 세션을 시작합니다.";
      }
    }

    // 채팅 렌더링
    const cc = document.getElementById('chat-content');
    let html = `<div style="text-align:center;padding:20px;color:var(--accent);font-weight:bold;font-size:1.4em;">${gState.session_title}</div>`;
    html += `<div class="bubble align-left"><div class="name-tag">PROLOGUE</div>${mdToSafeHtml(gState.prologue||"")}</div>`;

    (gState.ai_history||[]).forEach(m => {
      if(m.startsWith("**AI**:")){
        const text = m.replace("**AI**:","").trim();
        html += `<div class="bubble align-left"><div class="name-tag">AI</div>${mdToSafeHtml(text)}</div>`;
      } else if(m.startsWith("**Round**:")){
        const raw = m.replace("**Round**:", "").trim();
        const parts = raw.split(" / ");
        parts.forEach(p => {
          const sep = p.indexOf(":");
          if(sep > -1){
            const name = p.substring(0, sep).trim();
            const body = p.substring(sep+1).trim();
            const myProfileName = gState.profiles[myRole]?.name;
            const isMe = (name === myProfileName);
            const alignClass = isMe ? "align-right" : "align-left";
            html += `<div class="bubble ${alignClass}"><div class="name-tag">${name}</div>${mdToSafeHtml(body)}</div>`;
          } else {
            html += `<div class="bubble align-left">${mdToSafeHtml(p)}</div>`;
          }
        });
      }
    });
    cc.innerHTML = html;
    document.getElementById('chat-window').scrollTop = document.getElementById('chat-window').scrollHeight;

    // 내 입력 대기(고스트)
    const myPending = gState.pending_inputs?.[myRole];
    if(myPending && myPending.text){
      const myName = gState.profiles[myRole].name;
      cc.innerHTML += `<div class="bubble align-right" style="opacity:0.7;"><div class="name-tag">${myName}</div>${mdToSafeHtml(myPending.text)}</div>`;
    }

    // 상태 바 업데이트
    const pends = (gState.pending_status||[]);
    const typers = (gState.typing_status||[]);
    const other = (myRole==='user1')?'user2':'user1';
    const otherName = gState.profiles?.[other]?.name || "상대";

    let st = "대기 중...";
    if(typers.includes(other)) st = `<span class="typing-anim">${otherName} 입력 중...</span>`;
    else if(pends.includes(other)) st = `✅ ${otherName} 입력 완료`;
    if(pends.includes(myRole)) st += " / 나도 완료";
    document.getElementById('status').innerHTML = st;

    const myDone = pends.includes(myRole);
    sendBtn.disabled = myDone || !gState.session_started || myRole==='readonly';
    skipBtn.disabled = myDone || !gState.session_started || myRole==='readonly';

    // 프로필 복원 (카운터 동기화 포함)
    const p = (myRole && gState.profiles && gState.profiles[myRole]) ? gState.profiles[myRole] : {name:"",bio:"",canon:"",locked:false};
    const activeId = document.activeElement?.id || "";

    if(activeId!=='p-name') document.getElementById('p-name').value = p.name || "";
    if(activeId!=='p-bio') {
      document.getElementById('p-bio').value = p.bio || "";
      upCnt(document.getElementById('p-bio'));
    }
    if(activeId!=='p-canon') {
      document.getElementById('p-canon').value = p.canon || "";
      upCnt(document.getElementById('p-canon'));
    }

    const locked = !!p.locked;
    const overlay = document.getElementById('profile-lock-overlay');
    if(overlay) overlay.style.display = locked ? 'block' : 'none';

    const disableProfile = (myRole==='readonly') || locked;
    document.getElementById('p-name').readOnly = disableProfile;
    document.getElementById('p-name').disabled = disableProfile;
    document.getElementById('p-bio').disabled = disableProfile;
    document.getElementById('p-canon').disabled = disableProfile;

    const rb = document.getElementById('ready-btn');
    rb.disabled = disableProfile;
    rb.innerText = locked ? "설정이 확정되었습니다" : "설정 저장";

    // 마스터/예시/키워드 복원
    if(activeId!=='m-title') document.getElementById('m-title').value = gState.session_title || "";
    if(activeId!=='m-sys') { document.getElementById('m-sys').value = gState.sys_prompt || ""; upCnt(document.getElementById('m-sys')); }
    if(activeId!=='m-pro') { document.getElementById('m-pro').value = gState.prologue || ""; upCnt(document.getElementById('m-pro')); }
    if(activeId!=='m-sum') document.getElementById('m-sum').value = gState.summary || "";
    document.getElementById('m-ai-model').value = gState.ai_model || "gpt-5.2";
    document.getElementById('m-solo').value = gState.solo_mode ? "true" : "false";

    if(gState.examples){
      for(let i=0;i<3;i++){
        const ex = gState.examples[i] || {};
        if(activeId!==`ex-q-${i}`) document.getElementById(`ex-q-${i}`).value = ex.q || "";
        if(activeId!==`ex-a-${i}`) document.getElementById(`ex-a-${i}`).value = ex.a || "";
      }
    }
    if(activeId!=='kw-c') upCnt(document.getElementById('kw-c'));

    renderLoreList();
  }

  function send(){
    const t = document.getElementById('msg-input').value.trim();
    if(!t) return;

    // 버튼 잠금 (중복 전송 방지)
    document.getElementById('send-btn').disabled = true;
    document.getElementById('msg-input').disabled = true;

    socket.emit('client_message', {uid: myRole, text: t});
    document.getElementById('msg-input').value='';
    socket.emit('stop_typing', {uid: myRole});
  }
  function skipTurn(){
    if(!confirm("스킵?")) return;
    socket.emit('skip_turn', {uid: myRole});
    socket.emit('stop_typing', {uid: myRole});
  }

  function saveProfile(){
    const name = document.getElementById('p-name').value;
    if(!name) return alert("이름 필수");
    if(confirm("확정하시겠습니까? (수정 불가)")){
      socket.emit('update_profile', {
        uid: myRole,
        name,
        bio: document.getElementById('p-bio').value,
        canon: document.getElementById('p-canon').value
      });
    }
  }

  function saveMaster(){
    socket.emit('save_master_base', {
      title: document.getElementById('m-title').value,
      sys: document.getElementById('m-sys').value,
      pro: document.getElementById('m-pro').value,
      sum: document.getElementById('m-sum').value,
      model: document.getElementById('m-ai-model').value,
      solo_mode: (document.getElementById('m-solo').value === "true")
    });
    alert("저장됨");
  }

  function startSession(){ socket.emit('start_session'); }
  function sessionReset(){
    if(!confirm("정말로 초기화하시겠습니까?")) return;
    const pw = prompt("관리자 비밀번호를 입력하세요:");
    if(!pw) return;

    socket.emit('reset_session', {password: pw});

    // 화면 즉시 청소 (UX 향상)
    try{
      document.getElementById('chat-content').innerHTML = "";
      document.getElementById('msg-input').value = "";

      const pn = document.getElementById('p-name'); if(pn) pn.value = "";
      const pb = document.getElementById('p-bio'); if(pb) { pb.value = ""; upCnt(pb); }
      const pc = document.getElementById('p-canon'); if(pc) { pc.value = ""; upCnt(pc); }

      const mt = document.getElementById('m-title'); if(mt) mt.value = "";
      const ms = document.getElementById('m-sys'); if(ms) { ms.value = ""; upCnt(ms); }
      const mp = document.getElementById('m-pro'); if(mp) { mp.value = ""; upCnt(mp); }
      const msum = document.getElementById('m-sum'); if(msum) msum.value = "";

      for(let i=0;i<3;i++){
        const q = document.getElementById(`ex-q-${i}`); if(q) q.value="";
        const a = document.getElementById(`ex-a-${i}`); if(a) a.value="";
      }
      clearLoreEditor();
    } catch(e){}

    const modal = document.getElementById('admin-modal');
    if(modal) modal.style.display = 'none';
  }

  function addLoreWithTags(){
    const title = document.getElementById('kw-t').value;
    const content = document.getElementById('kw-c').value;
    const triggers = document.getElementById('tag-hidden').value;
    const idx = parseInt(document.getElementById('kw-index').value);
    if(!title) return alert("이름 필요");
    if(!triggers) return alert("트리거 필요");
    socket.emit('add_lore', {title, triggers, content, index: idx});
    clearLoreEditor();
  }

  function editLore(i){
    const l = gState.lorebook[i];
    document.getElementById('kw-t').value = l.title || "";
    document.getElementById('kw-c').value = l.content || "";
    document.getElementById('kw-index').value = i;
    loadTagsFromString(l.triggers || "");
    upCnt(document.getElementById('kw-c'));
  }
  function delLore(i){ socket.emit('del_lore', {index:i}); }

  function renderLoreList(){
    const list = document.getElementById('lore-list');
    if(!gState || !gState.lorebook) return;
    list.innerHTML = gState.lorebook.map((l,i)=>`
      <div class="lore-row" data-index="${i}">
        <div class="drag-handle">☰</div>
        <div class="lore-main">
          <div class="lore-title">${l.title}</div>
          <div class="lore-trg">${l.triggers}</div>
        </div>
        <div class="lore-actions">
          <button class="mini-btn mini-edit" onclick="editLore(${i})">수정</button>
          <button class="mini-btn mini-del" onclick="delLore(${i})">삭제</button>
        </div>
      </div>
    `).join('');
    if(sortable) sortable.destroy();
    sortable = new Sortable(list, {
      handle: '.drag-handle',
      animation: 120,
      onEnd: (evt) => {
        if(evt.oldIndex === evt.newIndex) socket.emit('reorder_lore', {from: evt.oldIndex, to: evt.newIndex});
      }
    });
  }

  function uploadSessionFile(input){
    if(!input.files || !input.files[0]) return;
    const formData = new FormData();
    formData.append('file', input.files[0]);
    fetch('/import',{method:'POST',body:formData})
      .then(res=>{ if(res.ok) alert("복원됨"); else alert("실패"); input.value=''; })
      .catch(err=>alert("오류: "+err));
  }
</script>
</body>
</html>
"""

if __name__ == "__main__":
    try:
        import subprocess
        subprocess.run(["pkill", "-9", "ngrok"])
        ngrok.kill()

        public_url = ngrok.connect(5000).public_url
        print(f"🚀 [드림놀이] 서버 시작\n🔗 접속 주소: {public_url}")

        socketio.run(app, host="0.0.0.0", port=5000, allow_unsafe_werkzeug=True)
    except Exception as e: print(f"❌ 오류: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 3.2 MB/s eta 0:00:00
Mounted at /content/drive



🚀 서버가 시작되었습니다.
🔗 접속 주소: https://unadhesively-prolix-deb.ngrok-free.dev

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Dec/2025 06:25:16] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Dec/2025 06:25:18] "GET /socket.io/?EIO=4&transport=polling&t=PjPLJzp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Dec/2025 06:25:19] "POST /socket.io/?EIO=4&transport=polling&t=PjPLKLf&sid=NVQYE473wKth32FSAAAA HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Dec/2025 06:25:19] "GET /socket.io/?EIO=4&transport=polling&t=PjPLKLh&sid=NVQYE473wKth32FSAAAA HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Dec/2025 06:25:20] "GET /socket.io/?EIO=4&transport=polling&t=PjPLKXb&sid=NVQYE473wKth32FSAAAA HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Dec/2025 06:25:20] "POST /socket.io/?EIO=4&transport=polling&t=PjP